In [1]:
import os

In [2]:
%pwd

'd:\\github\\Github\\Text-Summarization\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\github\\Github\\Text-Summarization'

In [15]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    tokenized_data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenized_data_path=config.tokenized_data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
        )

        return model_evaluation_config

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
from evaluate import load

import torch
import pandas as pd
from tqdm import tqdm

d:\github\Github\Text-Summarization\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-09-28 11:02:05,409: INFO: config: PyTorch version 2.4.1 available.]
[2024-09-28 11:02:05,419: INFO: config: TensorFlow version 2.17.0 available.]


In [9]:
# class ModelEvaluation:
#     def __init__(self, config: ModelEvaluationConfig):
#         self.config = config


#     def generate_batch_sized_chunks(self,list_of_elements, batch_size):
#         """split the dataset into smaller batches that we can process simultaneously
#         Yield successive batch-sized chunks from list_of_elements."""
#         for i in range(0, len(list_of_elements), batch_size):
#             yield list_of_elements[i : i + batch_size]

    
#     def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
#                                batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
#                                column_text="article", 
#                                column_summary="highlights"):
#         article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
#         target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

#         for article_batch, target_batch in tqdm(
#             zip(article_batches, target_batches), total=len(article_batches)):
            
#             inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
#                             padding="max_length", return_tensors="pt")
            
#             summaries = model.generate(input_ids=inputs["input_ids"].to(device),
#                             attention_mask=inputs["attention_mask"].to(device), 
#                             length_penalty=0.8, num_beams=8, max_length=128)
#             ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
#             # Finally, we decode the generated texts, 
#             # replace the  token, and add the decoded texts with the references to the metric.
#             decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
#                                     clean_up_tokenization_spaces=True) 
#                 for s in summaries]      
            
#             decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
#             metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
#         #  Finally compute and return the ROUGE scores.
#         score = metric.compute()
#         return score


#     def evaluate(self):
#         device = "cuda" if torch.cuda.is_available() else "cpu"
#         tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
#         model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
#         #loading data 
#         dataset_samsum_pt = load_from_disk(self.config.data_path)
#         print("columns name: ",dataset_samsum_pt['test'].column_names)
#         print("input_ids: ",dataset_samsum_pt['test']["input_ids"])
#         print("attention_mask: ",dataset_samsum_pt['test']["attention_mask"])
#         print("labels: ",dataset_samsum_pt['test']["labels"])
#         rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
#         rouge_metric = load('rouge')

#         score = self.calculate_metric_on_test_ds(
#                 dataset_samsum_pt['test'][0:10], 
#                 rouge_metric, model_pegasus, 
#                 tokenizer, batch_size = 2, 
#                 # column_text = 'dialogue', 
#                 column_text = 'input_ids', 
#                 # column_summary= 'summary'
#                 column_summary= 'labels'
#             )

#         rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

#         df = pd.DataFrame(rouge_dict, index = ['pegasus'])
#         df.to_csv(self.config.metric_file_name, index=False)

In [19]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        # Try to load the dataset
        try:
            # First, try to load as a directory
            if os.path.isdir(self.config.data_path):
                dataset_samsum = load_from_disk(self.config.data_path)
                print(f"Loaded dataset from disk: {self.config.data_path}")
            else:
                # If not a directory, try to load as a JSON file
                dataset_samsum = load_dataset("json", data_files=self.config.data_path)
                print(f"Loaded dataset from JSON: {self.config.data_path}")
        except Exception as e:
            print(f"Error loading dataset: {e}")
            print("Please check if the dataset path is correct and the dataset exists.")
            return

        # Print dataset info for debugging
        print("Dataset structure:", dataset_samsum)
        print("Dataset columns:", dataset_samsum['test'].column_names)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = load('rouge')

        # Determine the correct column names
        text_column = 'dialogue' if 'dialogue' in dataset_samsum['test'].column_names else 'input_ids'
        summary_column = 'summary' if 'summary' in dataset_samsum['test'].column_names else 'labels'

        score = self.calculate_metric_on_test_ds(
                dataset_samsum['test'][0:10], 
                rouge_metric, model_pegasus, 
                tokenizer, batch_size = 2, 
                column_text = text_column, 
                column_summary = summary_column
            )

        rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index = ['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

    # The calculate_metric_on_test_ds method remains the same as in your original code

In [20]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-09-28 11:15:40,759: INFO: common: yaml file: config\config.yaml loaded succesfully.]
[2024-09-28 11:15:40,764: INFO: common: yaml file: params.yaml loaded succesfully.]
[2024-09-28 11:15:40,767: INFO: common: Created directory at: artifacts]
[2024-09-28 11:15:40,769: INFO: common: Created directory at: artifacts/model_evaluation]
Loaded dataset from disk: artifacts/data_ingestion/samsum_dataset
Dataset structure: DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})
Dataset columns: ['id', 'dialogue', 'summary']


100%|██████████| 5/5 [05:26<00:00, 65.27s/it]


[2024-09-28 11:21:18,512: INFO: rouge_scorer: Using default tokenizer.]
